In [40]:
from datasets import ligands
from commons.process_mols import read_molecules_from_sdf, get_lig_graph_revised
from equibind_inference import get_default_args, parse_arguments
import importlib
from torch.utils.data import DataLoader
importlib.reload(ligands)
import equibind_inference
importlib.reload(equibind_inference)
import dgl
from dgl import batch
import torch
from commons.utils import seed_all
import copy
import argparse
importlib.reload(argparse)

<module 'argparse' from '/home/amfaber/anaconda3/envs/equibind/lib/python3.7/argparse.py'>

In [41]:
args, cmdline = parse_arguments(["-r", "/home/amfaber/Masters/POR-DD/data/raw_data/cyp_screen/protein.pdb",
                        "-l", "/home/amfaber/Masters/POR-DD/data/raw_data/cyp_screen/test_3D_opt_1216.sdf"])
args = get_default_args(args, cmdline)

In [42]:
ligs, rec_graph, model = equibind_inference.load_statics(args)
rec_graph = rec_graph.to(args.device)

device = cuda:0


RDKit WARNING: [19:10:58] Warning: molecule is tagged as 3D, but all Z coords are zero
[19:10:58] Warning: molecule is tagged as 3D, but all Z coords are zero


In [43]:
idx = 1050
bs = 8
lig_data = ligands.Ligands(ligs[(idx//bs)*bs:((idx//bs)+1)*bs], rec_graph, args, n_jobs = 1)
# lig_data = ligands.Ligands(ligs, rec_graph, args, n_jobs = 4)


0it [00:00, ?it/s][Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
8it [00:00, 52.06it/s]
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.2s finished


[2022-05-22 19:11:02.996523] The lig_radius 5 was too small for one lig atom such that it had no neighbors. So we connected 3 to the closest other lig atom [0]


0it [00:00, ?it/s][Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
8it [00:00, 407.24it/s]
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished


In [44]:
lig_loader = DataLoader(lig_data, batch_size = bs, collate_fn = lig_data.collate(rec_batch = True))

In [46]:
batch = next(iter(lig_loader))
ligs, lig_coords, lig_graphs, rec_graphs, geometry_graphs = batch
with torch.no_grad():
    pred, suc, fail = equibind_inference._run(next(iter(lig_loader)), model)
for lig, lig_coord, lig_graph, rec_graph, geometry_graph in batch:
    equibind_inference.run_corrections(lig, lig_coord, pred)

[2022-05-22 19:11:14.392611] None complex_names where Nan encountered
Succeeded for Thonzylamine
Succeeded for Cyclomethicone 5
Succeeded for Hexylresorcinol
[2022-05-22 19:11:15.190677] None complex_names where Nan encountered
Failed for Chromium picolinate
Succeeded for Methyl undecenoyl leucinate
Succeeded for DL-Methylephedrine
Succeeded for Diethyltoluamide
Succeeded for Vanillyl butyl ether


DGLError: Invalid key "0". Must be one of the edge types.

In [ ]:
pred[2]

tensor([[17.9542,  3.2036, 62.2651],
        [17.3889,  3.2567, 62.3026],
        [16.7102,  3.6463, 62.3573],
        [15.8822,  3.9909, 63.0453],
        [15.4047,  4.2120, 63.0222],
        [14.5473,  4.6452, 63.3473],
        [13.4776,  4.9445, 64.8004],
        [12.9852,  5.1882, 65.6492],
        [13.4660,  5.8248, 65.5774],
        [14.0161,  5.0252, 64.2333],
        [12.8471,  5.6899, 66.3143],
        [13.9480,  5.6137, 64.7232],
        [12.4853,  6.0445, 67.0912],
        [13.9514,  5.8752, 64.6366],
        [18.5769,  3.2336, 62.2478],
        [17.8971,  3.3200, 62.3166],
        [17.4279,  3.4691, 62.5217],
        [17.7210,  3.2649, 62.3602],
        [17.4294,  3.3732, 62.2524],
        [16.6550,  3.7203, 62.6537],
        [16.4544,  3.9434, 62.5628],
        [15.8046,  4.0962, 63.0771],
        [15.9222,  4.0276, 62.9092],
        [15.5524,  4.1002, 63.0780],
        [15.1116,  4.3068, 63.2008],
        [14.6416,  4.6702, 63.5371],
        [14.3754,  4.6448, 63.2891],
 

In [ ]:
torch.cuda.empty_cache()
for i, sample_batched in enumerate(lig_loader):
    with torch.no_grad():
        batched_res = model(*sample_batched[2:], complex_names = [mol.GetProp('_Name') for mol in sample_batched[0]])[0]
    

invoked. shape = torch.Size([165, 17])
invoked. shape = torch.Size([6212, 3])
[2022-05-22 18:22:18.294238] ['Thonzylamine', 'Cyclomethicone 5', 'Hexylresorcinol', 'Chromium picolinate'] complex_names where Nan encountered


AssertionError: 

In [9]:
with torch.no_grad():
    _ = model(*sample_batched[2:], complex_names = [mol.GetProp('_Name') for mol in sample_batched[0]])[0]

invoked. shape = torch.Size([165, 17])
invoked. shape = torch.Size([6212, 3])
[2022-05-22 18:22:20.813819] ['Thonzylamine', 'Cyclomethicone 5', 'Hexylresorcinol', 'Chromium picolinate'] complex_names where Nan encountered


AssertionError: 

In [13]:
with torch.no_grad(), open("test", "w+") as file:
    pass

In [10]:
sample_batched[2].ndata['feat'].shape

torch.Size([165, 17])

In [11]:
sample_batched[3].ndata['feat'].shape

torch.Size([6212, 3])

In [20]:
with test.local_scope():
    x = torch.tensor([1]*3)
    test.ndata['feat'] = x
    x[0] = 3
    print(test.ndata['feat'])
print(test.ndata['feat'])

tensor([3, 1, 1])
tensor([0, 0, 0])


In [50]:
AtomEncoder(lig_data[0][2])

Graph(num_nodes=26, num_edges=412,
      ndata_schemes={'feat': Scheme(shape=(17,), dtype=torch.float32), 'x': Scheme(shape=(3,), dtype=torch.float32), 'mu_r_norm': Scheme(shape=(5,), dtype=torch.float32), 'new_x': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'feat': Scheme(shape=(15,), dtype=torch.float32)})

In [48]:
sample_batched[2].ndata['feat'].shape

torch.Size([26, 64])

In [46]:
lig_data[0][2].ndata['new_x'] == sample_batched[2].ndata['new_x']

tensor([[True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True]], device='cuda:0')

In [12]:
test is sample_batched[2].ndata['feat']

True

In [22]:
sample_batched[2].ndata['feat'] = torch.tensor([1]*26).to(args.device)

In [24]:
sample_batched[2].ndata['feat']

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1], device='cuda:0')

In [23]:
test

tensor([[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        [-0.0213, -0.7755,  0.2933,  ..., -0.1179, -0.6421, -0.5562],
        [-0.5568, -0.4912, -0.0321,  ...,  0.3873, -0.2955, -0.1874],
        ...,
        [-0.2246, -0.1478, -0.2877,  ...,  0.4583, -0.5429, -0.2579],
        [-0.3535, -0.0949, -0.3343,  ...,  0.4454, -0.5163, -0.3957],
        [ 0.3798, -0.2484,  0.2378,  ..., -0.5562, -0.3614, -0.4164]],
       device='cuda:0')

In [15]:
with torch.no_grad():
    model(*sample_batched[2:], complex_names = [mol.GetProp('_Name') for mol in sample_batched[0]])[0]

invoked. shape = torch.Size([314, 17])
invoked. shape = torch.Size([12424, 64])


AssertionError: x.shape[1] = 64, self.num_categorical_features =  1, self.num_scalar_features = 2

In [11]:
ligs, lig_coords, lig_graphs, rec_graphs, geometry_graphs = sample_batched

In [15]:
len(list(map(dgl.unbatch, [lig_graphs, rec_graphs, geometry_graphs]))[0])

16

In [50]:
lig_data[0][2].ndata["feat"]

tensor([[ 5.0000,  0.0000,  4.0000,  ...,  0.0000,  0.0000, -0.0416],
        [ 5.0000,  0.0000,  4.0000,  ...,  0.0000,  0.0000,  0.0437],
        [ 7.0000,  0.0000,  2.0000,  ...,  0.0000,  0.0000, -0.3814],
        ...,
        [ 0.0000,  0.0000,  1.0000,  ...,  0.0000,  0.0000,  0.0254],
        [ 0.0000,  0.0000,  1.0000,  ...,  0.0000,  0.0000,  0.0254],
        [ 0.0000,  0.0000,  1.0000,  ...,  0.0000,  0.0000,  0.0254]],
       device='cuda:0')

In [42]:
lig_data[0][2].ndata["feat"].long()

tensor([[14,  0,  4,  5,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 7,  0,  2,  5,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 7,  0,  2,  5,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 7,  0,  2,  5,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 7,  0,  2,  5,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 7,  0,  1,  5,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 7,  0,  1,  5,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 6,  0,  2,  5,  0,  0,  0,  1,  1,  1,  0,  0,  0,  1,  0,  0,  0],
        [ 5,  0,  3,  5,  0,  0,  0,  1,  1,  1,  0,  0,  0,  1,  0,  0,  0],
        [ 5,  0,  3,  5,  0,  0,  0,  1,  1,  1,  0,  0,  0,  1,  0,  0,  0],
        [ 5,  0,  4,  5,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 5,  0,  3,  5,  0,  0,  0,  1,  1,  1,  0,  0,  0,  1,  0,  0,  0],
        [ 5,  0,  3,  5,  0,  0,  0,  1,  1,  1,  0,  0,  0,  1,

In [31]:
model(*sample_batched[2:], complex_names = [mol.GetProp('_Name') for mol in sample_batched[0]])[0]

AssertionError: x.shape[1] = 64, self.num_categorical_features =  16, self.num_scalar_features = 1

In [10]:
1200/(10*16)*8.1

60.75

In [35]:
it

In [198]:
len(sample_batched[1])

8

In [66]:
torch.cuda.memory_reserved()>>20

800

In [35]:
torch.cuda.empty_cache()

In [70]:
torch.cuda.memory_reserved()//(1<<20)

800

In [65]:
1<<10

1024

In [180]:
sample_batched = next(iter(batched_rec))
sample_single = next(iter(single_rec))

seed_all(1)
with torch.no_grad():
    single_res = model(*sample_single[1:])[:-1]

seed_all(1)
with torch.no_grad():
    batched_res = model(*sample_batched[1:])[0]

lig_graphs = [copy.deepcopy(graph) for graph in res[:bs][1]]
geometry_graphs = [graph for graph in res[:bs][3]]
seed_all(1)
manual_res = None
with torch.no_grad():
    for lig_graph, geometry_graph in zip(lig_graphs, geometry_graphs):
        if manual_res == None:
            manual_res = model(lig_graph, rec_graph, geometry_graph)[:-1]
        else:
            new = model(lig_graph, rec_graph, geometry_graph)[:-1]
            for ele, n in zip(manual_res, new):
                ele += n


In [188]:
sample_batched[1].ndata["new_x"]

tensor([[ 2.9340e+00, -1.2581e+00, -2.1088e+00],
        [ 2.0240e+00, -7.6276e-01, -7.5346e-01],
        [-2.3342e+00,  8.3341e-01,  1.8964e+00],
        [ 1.9232e+00, -1.9486e+00, -3.2786e+00],
        [ 3.7085e+00,  9.1072e-02, -2.7734e+00],
        [-8.3063e-01, -4.6569e-01, -1.5841e+00],
        [ 3.9802e+00, -2.2665e+00, -1.6897e+00],
        [ 3.2115e-02, -1.5851e+00,  3.2719e+00],
        [ 4.7312e-01, -1.4434e+00,  9.1992e-01],
        [-5.5711e-01, -4.7118e-01,  7.9440e-01],
        [ 1.3309e+00, -1.8792e+00, -2.4594e-01],
        [-1.2809e+00, -8.6633e-02,  1.9500e+00],
        [-9.5441e-01, -6.5748e-01,  3.1903e+00],
        [ 7.3798e-01, -1.9858e+00,  2.1870e+00],
        [-1.6999e+00, -2.7394e-01,  4.4321e+00],
        [-8.5791e-01,  1.7945e-01, -5.0359e-01],
        [-2.6991e+00,  1.2206e+00,  1.0366e+00],
        [ 1.2718e+00, -1.2365e+00, -3.5006e+00],
        [ 2.9786e+00,  7.3487e-01, -2.9571e+00],
        [ 6.7291e-01, -2.3418e+00, -1.0136e+00],
        [ 2.0553e+00

In [183]:
equibind_inference.run_corrections(ligs[0], sample_batched[], batched_res[0])

tensor([[-3.8453e-02,  1.6737e+01,  7.3787e+01],
        [-3.1174e-01,  1.6831e+01,  7.2777e+01],
        [-1.3523e+00,  1.8286e+01,  6.8686e+01],
        [ 4.0060e-02,  1.6786e+01,  7.4727e+01],
        [-2.6052e-01,  1.6274e+01,  7.5340e+01],
        [-1.0196e+00,  1.8157e+01,  7.1608e+01],
        [ 2.7455e-01,  1.6196e+01,  7.4765e+01],
        [-1.0560e-01,  1.6952e+01,  6.8890e+01],
        [-1.6517e-01,  1.7229e+01,  7.0708e+01],
        [-5.9970e-01,  1.7700e+01,  7.0311e+01],
        [ 8.4728e-02,  1.6993e+01,  7.2016e+01],
        [-8.2748e-01,  1.7821e+01,  6.9292e+01],
        [-5.6004e-01,  1.7414e+01,  6.8733e+01],
        [-3.5707e-02,  1.7011e+01,  7.0029e+01],
        [-8.5652e-01,  1.7504e+01,  6.7340e+01],
        [-9.1743e-01,  1.8024e+01,  7.0912e+01],
        [-1.6051e+00,  1.8616e+01,  6.9034e+01],
        [-4.4526e-02,  1.6816e+01,  7.4173e+01],
        [-2.0862e-01,  1.6685e+01,  7.4627e+01],
        [-1.2859e-01,  1.6877e+01,  7.2354e+01],
        [-6.9523e-02

In [138]:
for single_coords, batched_coords, manual_coords in zip(single_res[0], batched_res[0], manual_res[0]):
    print(((single_coords - batched_coords)**2).sum())
print()

for single_coords, batched_coords, manual_coords in zip(single_res[0], batched_res[0], manual_res[0]):
    print(((single_coords - manual_coords)**2).sum())
print()

for single_coords, batched_coords, manual_coords in zip(single_res[0], batched_res[0], manual_res[0]):
    print(((batched_coords - manual_coords)**2).sum())
print()


tensor(763.0118, device='cuda:0')
tensor(1519.2156, device='cuda:0')
tensor(92.5205, device='cuda:0')
tensor(4293.4443, device='cuda:0')
tensor(241.7461, device='cuda:0')
tensor(70.6493, device='cuda:0')
tensor(924.4200, device='cuda:0')
tensor(8494.0371, device='cuda:0')

tensor(762.9764, device='cuda:0')
tensor(1519.2283, device='cuda:0')
tensor(92.5231, device='cuda:0')
tensor(4363.1582, device='cuda:0')
tensor(234.4151, device='cuda:0')
tensor(70.6483, device='cuda:0')
tensor(935.2437, device='cuda:0')
tensor(8494.0342, device='cuda:0')

tensor(4.1634e-07, device='cuda:0')
tensor(2.6505e-08, device='cuda:0')
tensor(3.1971e-08, device='cuda:0')
tensor(4.2061, device='cuda:0')
tensor(2.1525, device='cuda:0')
tensor(7.0997e-09, device='cuda:0')
tensor(2.0750, device='cuda:0')
tensor(1.4599e-08, device='cuda:0')



In [173]:
a, _, _ = 1, 2, 3

In [167]:
idk[0][0]

tensor([[ 6.7998e-01,  1.3429e+01,  7.7769e+01],
        [ 4.3621e-01,  1.3325e+01,  7.6852e+01],
        [-7.8746e-01,  1.4115e+01,  7.2590e+01],
        [ 5.4453e-01,  1.3465e+01,  7.8758e+01],
        [ 5.8788e-01,  1.2879e+01,  7.9418e+01],
        [-7.4408e-01,  1.3959e+01,  7.5475e+01],
        [ 1.1868e+00,  1.3157e+01,  7.8825e+01],
        [ 7.5035e-01,  1.3861e+01,  7.3015e+01],
        [ 4.7933e-01,  1.3901e+01,  7.4689e+01],
        [-1.0091e-01,  1.4020e+01,  7.4269e+01],
        [ 7.0891e-01,  1.3852e+01,  7.6073e+01],
        [-2.6758e-01,  1.4063e+01,  7.3298e+01],
        [ 1.9994e-01,  1.3943e+01,  7.2787e+01],
        [ 6.6217e-01,  1.3870e+01,  7.4021e+01],
        [ 4.0861e-02,  1.3919e+01,  7.1335e+01],
        [-5.4422e-01,  1.4061e+01,  7.4970e+01],
        [-1.1803e+00,  1.4177e+01,  7.2910e+01],
        [ 4.0868e-01,  1.3438e+01,  7.8318e+01],
        [ 3.6893e-01,  1.3131e+01,  7.8788e+01],
        [ 5.5394e-01,  1.3676e+01,  7.6457e+01],
        [ 6.4997e-01

In [10]:
seed_all(1)
with torch.no_grad():
    single = model(res[0][1], rec_graph, res[0][2])[0]

[2022-05-20 22:53:14.090991] [ Using Seed :  1  ]


In [11]:
single[0]

tensor([[-3.8455e-02,  1.6737e+01,  7.3787e+01],
        [-3.1174e-01,  1.6831e+01,  7.2777e+01],
        [-1.3523e+00,  1.8286e+01,  6.8686e+01],
        [ 4.0050e-02,  1.6786e+01,  7.4727e+01],
        [-2.6051e-01,  1.6274e+01,  7.5340e+01],
        [-1.0196e+00,  1.8157e+01,  7.1608e+01],
        [ 2.7455e-01,  1.6196e+01,  7.4765e+01],
        [-1.0557e-01,  1.6952e+01,  6.8890e+01],
        [-1.6517e-01,  1.7229e+01,  7.0708e+01],
        [-5.9971e-01,  1.7700e+01,  7.0311e+01],
        [ 8.4731e-02,  1.6993e+01,  7.2016e+01],
        [-8.2749e-01,  1.7821e+01,  6.9292e+01],
        [-5.6003e-01,  1.7414e+01,  6.8733e+01],
        [-3.5692e-02,  1.7011e+01,  7.0029e+01],
        [-8.5650e-01,  1.7504e+01,  6.7340e+01],
        [-9.1745e-01,  1.8024e+01,  7.0912e+01],
        [-1.6051e+00,  1.8616e+01,  6.9034e+01],
        [-4.4533e-02,  1.6816e+01,  7.4173e+01],
        [-2.0863e-01,  1.6685e+01,  7.4627e+01],
        [-1.2859e-01,  1.6877e+01,  7.2354e+01],
        [-6.9523e-02

In [13]:
idk[0][0]

tensor([[-3.8454e-02,  1.6737e+01,  7.3787e+01],
        [-3.1173e-01,  1.6831e+01,  7.2777e+01],
        [-1.3523e+00,  1.8286e+01,  6.8686e+01],
        [ 4.0051e-02,  1.6786e+01,  7.4727e+01],
        [-2.6051e-01,  1.6274e+01,  7.5340e+01],
        [-1.0196e+00,  1.8157e+01,  7.1608e+01],
        [ 2.7455e-01,  1.6196e+01,  7.4765e+01],
        [-1.0557e-01,  1.6952e+01,  6.8890e+01],
        [-1.6517e-01,  1.7229e+01,  7.0708e+01],
        [-5.9971e-01,  1.7700e+01,  7.0311e+01],
        [ 8.4733e-02,  1.6993e+01,  7.2016e+01],
        [-8.2749e-01,  1.7821e+01,  6.9292e+01],
        [-5.6003e-01,  1.7414e+01,  6.8733e+01],
        [-3.5690e-02,  1.7011e+01,  7.0029e+01],
        [-8.5650e-01,  1.7504e+01,  6.7340e+01],
        [-9.1745e-01,  1.8024e+01,  7.0912e+01],
        [-1.6051e+00,  1.8616e+01,  6.9034e+01],
        [-4.4532e-02,  1.6816e+01,  7.4173e+01],
        [-2.0862e-01,  1.6685e+01,  7.4627e+01],
        [-1.2859e-01,  1.6877e+01,  7.2354e+01],
        [-6.9521e-02

In [12]:
idk[0][0]-single[0]

tensor([[ 1.6689e-06, -9.5367e-06,  3.8147e-05],
        [ 1.6689e-06, -9.5367e-06,  3.0518e-05],
        [ 5.9605e-07, -1.1444e-05,  3.0518e-05],
        [ 1.4305e-06, -9.5367e-06,  3.8147e-05],
        [ 1.6689e-06, -9.5367e-06,  3.8147e-05],
        [ 5.9605e-07, -1.1444e-05,  3.8147e-05],
        [ 1.9073e-06, -9.5367e-06,  3.8147e-05],
        [ 1.9073e-06, -1.1444e-05,  3.0518e-05],
        [ 1.4305e-06, -9.5367e-06,  3.0518e-05],
        [ 9.5367e-07, -1.1444e-05,  3.0518e-05],
        [ 1.9073e-06, -1.1444e-05,  3.8147e-05],
        [ 1.1921e-06, -1.1444e-05,  3.0518e-05],
        [ 1.9073e-06, -1.1444e-05,  3.0518e-05],
        [ 1.6689e-06, -1.1444e-05,  3.0518e-05],
        [ 1.4305e-06, -1.1444e-05,  3.0518e-05],
        [ 3.5763e-07, -1.1444e-05,  3.0518e-05],
        [ 0.0000e+00, -1.1444e-05,  3.0518e-05],
        [ 1.6689e-06, -9.5367e-06,  3.0518e-05],
        [ 1.4305e-06, -9.5367e-06,  3.0518e-05],
        [ 1.6689e-06, -9.5367e-06,  3.0518e-05],
        [ 1.4305e-06